In [17]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.datasets.last_month_last_day import LMLDDataset
from hnmchallenge.constant import *

In [18]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [37]:
dataset = LMLDDataset()
dr = DataReader()

In [38]:
fd=dataset.get_full_data()

In [39]:
fd

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2
16020500,2020-09-22,214284,19647,0.042356,2
16020501,2020-09-22,837397,25890,0.043203,1
16020502,2020-09-22,424416,6429,0.006763,1


In [40]:
data=fd.groupby([DEFAULT_USER_COL])["t_dat"].max().reset_index()

In [41]:
max_date = fd["t_dat"].max() + pd.to_timedelta(1, unit="D")

In [35]:
max_date

Timestamp('2020-09-23 00:00:00')

In [42]:
data["t_diff"] = data["t_dat"].apply(lambda x: 1 / (max_date - x).days)

In [43]:
data

,customer_id,t_dat,t_diff
0,0,2020-04-15,0.006211
1,1,2020-08-12,0.023810
2,2,2020-03-10,0.005076
3,3,2019-09-28,0.002770
4,4,2018-09-20,0.001362
...,...,...,...
1167045,1167045,2020-09-22,1.000000
1167046,1167046,2020-09-22,1.000000
1167047,1167047,2020-09-22,1.000000
1167048,1167048,2020-09-22,1.000000


In [32]:
data

,customer_id,last_day
0,0,2020-04-15
1,1,2020-08-12
2,2,2020-03-10
3,3,2019-09-28
4,4,2018-09-20
...,...,...
1167045,1167045,2020-09-22
1167046,1167046,2020-09-22
1167047,1167047,2020-09-22
1167048,1167048,2020-09-22


In [28]:
data=pd.merge(fd,data,on=DEFAULT_USER_COL,how="left")

In [29]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,last_day
0,2018-09-20,0,0,0.008458,2,2020-04-15
1,2018-09-20,0,0,0.008458,2,2020-04-15
2,2018-09-20,1,1,0.033881,1,2020-08-12
3,2018-09-20,2,2,0.021593,1,2020-03-10
4,2018-09-20,3,3,0.022017,2,2019-09-28
...,...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2,2020-09-22
16020500,2020-09-22,214284,19647,0.042356,2,2020-09-22
16020501,2020-09-22,837397,25890,0.043203,1,2020-09-22
16020502,2020-09-22,424416,6429,0.006763,1,2020-09-22


In [13]:
df=data.groupby([DEFAULT_USER_COL]).count().reset_index()

In [14]:
df

,customer_id,t_dat,article_id,price,sales_channel_id
0,0,12,12,12,12
1,1,23,23,23,23
2,2,28,28,28,28
3,3,6,6,6,6
4,4,1,1,1,1
...,...,...,...,...,...
1167045,1167045,1,1,1,1
1167046,1167046,1,1,1,1
1167047,1167047,3,3,3,3
1167048,1167048,1,1,1,1


In [15]:
df=df[[DEFAULT_USER_COL, "price"]]

In [16]:
df

,customer_id,price
0,0,12
1,1,23
2,2,28
3,3,6
4,4,1
...,...,...
1167045,1167045,1
1167046,1167046,1
1167047,1167047,3
1167048,1167048,1


In [8]:
max_date

Timestamp('2020-09-22 00:00:00')

In [9]:
intervals = [
            (max_date - pd.to_timedelta(7, unit="D"),max_date),
            (max_date - pd.to_timedelta(14, unit="D"),max_date),
            (max_date - pd.to_timedelta(21, unit="D"),max_date),
            (max_date - pd.to_timedelta(28, unit="D"),max_date)
        ]

In [13]:
intervals[0]

AttributeError: 'tuple' object has no attribute 'str'

In [13]:
intervals1=[("2020-09-15","2020-09-08")]
m = np.logical_or.reduce(
            [np.logical_and(fd["t_dat"] >= l, fd["t_dat"] <= u) for l, u in intervals1]
        )
data_df = fd.loc[m]
duplicated_rows = data_df[
            data_df.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])
        ]
count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": "popularity"}
        )

In [14]:
feature

,article_id,popularity
0,4,8
1,5,8
2,6,2
3,7,127
4,8,45
...,...,...
10146,26234,1
10147,26240,1
10148,26243,1
10149,26245,1


In [20]:
df = pd.DataFrame(columns=[DEFAULT_ITEM_COL]) 
for idx, i in enumerate(intervals):
    m = np.logical_or.reduce(
            [np.logical_and(fd["t_dat"] >= i[0], fd["t_dat"] <= i[1]) ]
        )
    data_df = fd.loc[m]
    duplicated_rows = data_df[
            data_df.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])
        ]
    count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
    feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": f"popularity_{idx+1}"}
        )
    feature[f"popularity_{idx+1}"] = (
            feature[f"popularity_{idx+1}"] - feature[f"popularity_{idx+1}"].min()
        ) / (
            feature[f"popularity_{idx+1}"].max()
            - feature[f"popularity_{idx+1}"].min()
        )
    df=df.merge(feature, on=DEFAULT_ITEM_COL, how='outer')



In [21]:
df

,article_id,popularity_1,popularity_2,popularity_3,popularity_4
0,2,0.000000,0.000000,0.000000,0.000000
1,4,0.019868,0.019868,0.019868,0.019868
2,5,0.016556,0.016556,0.016556,0.016556
3,6,0.001656,0.001656,0.001656,0.001656
4,7,0.273179,0.273179,0.273179,0.273179
...,...,...,...,...,...
11545,26234,0.000000,0.000000,0.000000,0.000000
11546,26240,0.000000,0.000000,0.000000,0.000000
11547,26243,0.000000,0.000000,0.000000,0.000000
11548,26245,0.000000,0.000000,0.000000,0.000000


In [ ]:
data_df = fd[fd["t_dat"] >= "2020-09-01"]
duplicated_rows = data_df[
            data_df.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])
        ]
count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": "popularity"}
        )

In [77]:
last_month = fd[(fd["t_dat"] >= "2020-09-01")]
sorted_data = last_month.sort_values([DEFAULT_USER_COL, "t_dat"]).reset_index(
            drop=True
        )
sorted_data["last_buy"] = sorted_data.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)

In [78]:
sorted_data

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
0,2020-09-10,8,21060,0.016932,1,2020-09-10
1,2020-09-20,15,25551,0.016271,2,2020-09-20
2,2020-09-20,15,11032,0.011390,2,2020-09-20
3,2020-09-20,15,23239,0.065102,2,2020-09-20
4,2020-09-20,15,23276,0.032559,2,2020-09-20
...,...,...,...,...,...,...
798264,2020-09-22,1167049,9691,0.022017,2,2020-09-22
798265,2020-09-22,1167049,2216,0.016932,2,2020-09-22
798266,2020-09-22,1167049,997,0.032186,2,2020-09-22
798267,2020-09-22,1167049,16131,0.022017,2,2020-09-22


In [79]:
hold_out = sorted_data[(sorted_data['t_dat'] > (sorted_data["last_buy"]-pd.to_timedelta(7, unit='D'))) & (sorted_data['t_dat'] <=  sorted_data["last_buy"])]

In [80]:
hold_out

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
0,2020-09-10,8,21060,0.016932,1,2020-09-10
1,2020-09-20,15,25551,0.016271,2,2020-09-20
2,2020-09-20,15,11032,0.011390,2,2020-09-20
3,2020-09-20,15,23239,0.065102,2,2020-09-20
4,2020-09-20,15,23276,0.032559,2,2020-09-20
...,...,...,...,...,...,...
798264,2020-09-22,1167049,9691,0.022017,2,2020-09-22
798265,2020-09-22,1167049,2216,0.016932,2,2020-09-22
798266,2020-09-22,1167049,997,0.032186,2,2020-09-22
798267,2020-09-22,1167049,16131,0.022017,2,2020-09-22


In [81]:
hold_out[hold_out[DEFAULT_USER_COL]==31]

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
72,2020-09-17,31,19633,0.033881,2,2020-09-22
73,2020-09-17,31,19624,0.033881,2,2020-09-22
74,2020-09-17,31,19624,0.033881,2,2020-09-22
75,2020-09-17,31,25313,0.033881,2,2020-09-22
76,2020-09-17,31,11679,0.022017,2,2020-09-22
77,2020-09-20,31,21318,0.033881,2,2020-09-22
78,2020-09-22,31,22832,0.022017,2,2020-09-22
79,2020-09-22,31,10940,0.033881,2,2020-09-22
80,2020-09-22,31,23276,0.033881,2,2020-09-22
81,2020-09-22,31,24992,0.050831,2,2020-09-22


In [82]:
hold_in=sorted_data.loc[sorted_data.index.difference(hold_out.index), ]

In [83]:
hold_in

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
9,2020-09-01,25,3673,0.025407,2,2020-09-20
10,2020-09-01,25,20590,0.025407,2,2020-09-20
11,2020-09-01,25,8677,0.025407,2,2020-09-20
12,2020-09-01,25,18326,0.016932,2,2020-09-20
13,2020-09-01,25,4136,0.016932,2,2020-09-20
...,...,...,...,...,...,...
775686,2020-09-15,1160115,11588,0.022017,2,2020-09-22
775687,2020-09-15,1160115,2067,0.033881,2,2020-09-22
775688,2020-09-15,1160115,2067,0.033881,2,2020-09-22
775689,2020-09-15,1160115,3312,0.022017,2,2020-09-22


In [68]:
cond = sorted_data["t_dat"].isin(hold_out["t_dat"])


In [69]:

sorted_data.drop(sorted_data[cond].index, inplace=True)


In [70]:
sorted_data

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy


In [25]:
 hold_out = sorted_data[sorted_data["t_dat"] == sorted_data["last_buy"]]

,t_dat,customer_id,article_id,price,sales_channel_id,last_buy
9,2020-09-01,25,3673,0.025407,2,2020-09-20
10,2020-09-01,25,20590,0.025407,2,2020-09-20
11,2020-09-01,25,8677,0.025407,2,2020-09-20
12,2020-09-01,25,18326,0.016932,2,2020-09-20
13,2020-09-01,25,4136,0.016932,2,2020-09-20
...,...,...,...,...,...,...
730714,2020-09-01,1147309,21385,0.050831,2,2020-09-01
730715,2020-09-01,1147309,22522,0.050831,2,2020-09-01
730716,2020-09-01,1147310,7905,0.022864,1,2020-09-01
730717,2020-09-01,1147311,20448,0.033881,1,2020-09-01


In [5]:
fd

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2
16020500,2020-09-22,214284,19647,0.042356,2
16020501,2020-09-22,837397,25890,0.043203,1
16020502,2020-09-22,424416,6429,0.006763,1


In [32]:
last_2_month = fd[(fd["t_dat"] >= "2020-09-15")]

In [33]:
intervals = [
            ("2020-09-15", "2020-09-22")
        ]

In [34]:
m = np.logical_or.reduce(
            [np.logical_and(fd["t_dat"] >= l, fd["t_dat"] <= u) for l, u in intervals]
        )
last_week = fd.loc[m]

In [35]:
user=last_week.groupby([DEFAULT_USER_COL,"t_dat"]).count().reset_index()

In [36]:
user

,customer_id,t_dat,article_id,price,sales_channel_id
0,15,2020-09-20,6,6,6
1,25,2020-09-20,8,8,8
2,26,2020-09-21,2,2,2
3,30,2020-09-17,5,5,5
4,31,2020-09-15,14,14,14
...,...,...,...,...,...
85425,1167045,2020-09-22,2,2,2
85426,1167046,2020-09-22,1,1,1
85427,1167047,2020-09-22,3,3,3
85428,1167048,2020-09-22,2,2,2


In [37]:
user.groupby(DEFAULT_USER_COL).count()

,t_dat,article_id,price,sales_channel_id
customer_id,,,,
15,1,1,1,1
25,1,1,1,1
26,1,1,1,1
30,1,1,1,1
31,4,4,4,4
...,...,...,...,...
1167045,1,1,1,1
1167046,1,1,1,1
1167047,1,1,1,1


In [6]:
customers=dataset.get_customers_df()

In [7]:
customers=customers[[DEFAULT_USER_COL, "age"]]

In [8]:
customers

,customer_id,age
0,284440,49.0
1,26821,25.0
2,901651,24.0
3,1167050,54.0
4,1051706,52.0
...,...,...
1371975,470301,24.0
1371976,199407,21.0
1371977,111983,21.0
1371978,972397,18.0


In [9]:
fd2=pd.merge(fd, customers, on=DEFAULT_USER_COL, how='left')

In [15]:
fd3=fd2[fd2["age"]<=25]

In [25]:
fd3=fd2[fd2["age"]<=25].groupby([DEFAULT_ITEM_COL]).count().reset_index()


In [26]:
fd3

,article_id,t_dat,customer_id,price,sales_channel_id,age
0,0,1694,1694,1694,1694,1694
1,1,578,578,578,578,578
2,2,112,112,112,112,112
3,3,266,266,266,266,266
4,4,1610,1610,1610,1610,1610
...,...,...,...,...,...,...
24172,26237,1,1,1,1,1
24173,26238,1,1,1,1,1
24174,26243,3,3,3,3,3
24175,26245,2,2,2,2,2


In [27]:
fd3= fd3[[DEFAULT_ITEM_COL,"t_dat"]].rename({"t_dat": "pop_25"}, axis=1)

In [28]:
fd3["pop_25"] = (fd3["pop_25"] - fd3["pop_25"].min()) / (fd3["pop_25"].max()- fd3["pop_25"].min())

In [29]:
fd3

,article_id,pop_25
0,0,0.082280
1,1,0.028042
2,2,0.005395
3,3,0.012879
4,4,0.078198
...,...,...
24172,26237,0.000000
24173,26238,0.000000
24174,26243,0.000097
24175,26245,0.000049


In [30]:
fd4=fd2[(fd2["age"]>25) & (fd2["age"]<=40)].groupby([DEFAULT_ITEM_COL]).count().reset_index()

In [31]:
fd4= fd4[[DEFAULT_ITEM_COL,"t_dat"]].rename({"t_dat": "pop_25_40"}, axis=1)

In [32]:
fd4["pop_25_40"] = (fd4["pop_25_40"] - fd4["pop_25_40"].min()) / (fd4["pop_25_40"].max()- fd4["pop_25_40"].min())

In [33]:
fd4

,article_id,pop_25_40
0,0,0.076080
1,1,0.037570
2,2,0.027677
3,3,0.042956
4,4,0.162054
...,...,...
25662,26242,0.000000
25663,26247,0.000000
25664,26248,0.000000
25665,26249,0.000063


In [34]:
fd5=fd2[(fd2["age"]>40) & (fd2["age"]<=60)].groupby([DEFAULT_ITEM_COL]).count().reset_index()

In [35]:
fd5= fd5[[DEFAULT_ITEM_COL,"t_dat"]].rename({"t_dat": "pop_40_60"}, axis=1)

In [36]:
fd5["pop_40_60"] = (fd5["pop_40_60"] - fd5["pop_40_60"].min()) / (fd5["pop_40_60"].max()- fd5["pop_40_60"].min())

In [37]:
fd5

,article_id,pop_40_60
0,0,0.067828
1,1,0.090568
2,2,0.041650
3,3,0.077127
4,4,0.116355
...,...,...
25115,26230,0.000000
25116,26234,0.000078
25117,26235,0.000000
25118,26244,0.000000


In [42]:
fd6=fd2[fd2["age"]>60]

In [59]:
fd6 = (
            fd2[fd2["age"] > 60]
            .groupby([DEFAULT_ITEM_COL])
            .count()
            .reset_index()
        )

In [60]:
fd6= fd6[[DEFAULT_ITEM_COL,"t_dat"]].rename({"t_dat": "pop_60"}, axis=1)

In [61]:
fd6["pop_60"] = (fd6["pop_60"] - fd6["pop_60"].min()) / (fd6["pop_60"].max()- fd6["pop_60"].min())

In [62]:
fd6

,article_id,pop_60
0,0,0.025132
1,1,0.037698
2,2,0.068783
3,3,0.060185
4,4,0.095238
...,...,...
21257,26149,0.000661
21258,26162,0.000000
21259,26166,0.000000
21260,26191,0.000000


In [65]:
articles=dataset.get_full_data()
articles=articles[DEFAULT_ITEM_COL].drop_duplicates()

In [66]:
articles

0               0
2               1
3               2
4               3
6               4
            ...  
16017925    26247
16018130    26248
16018131    26249
16018396    26250
16019501    26251
Name: article_id, Length: 26252, dtype: int64

In [47]:
articles=pd.merge(fd2,articles,on=DEFAULT_ITEM_COL,how="left")

In [67]:
articles

,t_dat,customer_id,article_id,price,sales_channel_id,age
0,2018-09-20,0,0,0.008458,2,22.0
1,2018-09-20,0,0,0.008458,2,22.0
2,2018-09-20,1,1,0.033881,1,26.0
3,2018-09-20,2,2,0.021593,1,53.0
4,2018-09-20,3,3,0.022017,2,38.0
...,...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2,32.0
16020500,2020-09-22,214284,19647,0.042356,2,32.0
16020501,2020-09-22,837397,25890,0.043203,1,67.0
16020502,2020-09-22,424416,6429,0.006763,1,21.0


In [68]:
articles= articles[articles.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])]

In [71]:
feature=pd.merge(articles,fd3,on=[DEFAULT_ITEM_COL],how="left")
feature=pd.merge(feature,fd4,on=[DEFAULT_ITEM_COL],how="left")
feature=pd.merge(feature,fd5,on=[DEFAULT_ITEM_COL],how="left")
feature=pd.merge(feature,fd6,on=[DEFAULT_ITEM_COL],how="left")

In [72]:
feature=feature[[DEFAULT_ITEM_COL,"pop_25","pop_25_40","pop_40_60","pop_60"]]

In [69]:
feature=feature.drop_duplicates()

In [73]:
feature

,article_id,pop_25,pop_25_40,pop_40_60,pop_60
0,0,0.082280,0.076080,0.067828,0.025132
1,1,0.028042,0.037570,0.090568,0.037698
2,2,0.005395,0.027677,0.041650,0.068783
3,3,0.012879,0.042956,0.077127,0.060185
4,4,0.078198,0.162054,0.116355,0.095238
...,...,...,...,...,...
26247,26247,NaN,0.000000,NaN,NaN
26248,26248,NaN,0.000000,NaN,NaN
26249,26249,NaN,0.000063,NaN,NaN
26250,26250,NaN,NaN,0.000000,NaN


In [119]:
articles = dataset.get_articles_df()
articles = articles[[DEFAULT_ITEM_COL, "product_type_no"]]

In [120]:
fd=dataset.get_full_data()

In [121]:
# take the price on the last available day
last_date_item = (
            fd.groupby(DEFAULT_ITEM_COL)[["t_dat"]].max().reset_index()
        )
fd = pd.merge(
            fd, last_date_item, on=[DEFAULT_ITEM_COL, "t_dat"]
        ).drop_duplicates(subset=DEFAULT_ITEM_COL)[[DEFAULT_ITEM_COL, "price"]]


In [122]:
fd

,article_id,price
0,1859,0.061000
1,4060,0.016932
2,457,0.011847
3,8839,0.018627
4,5072,0.015237
...,...,...
54901,16131,0.022017
54902,15180,0.059305
54903,3590,0.010153
54904,17935,0.025407


In [147]:
fd2=pd.merge(fd, articles, on=DEFAULT_ITEM_COL, how='left')

In [148]:
fd2[fd2["product_type_no"]==-1]

,article_id,price,product_type_no
1208,18013,0.010831,-1
2231,6529,0.038966,-1
2497,7654,0.022017,-1
2827,18266,0.016932,-1
3203,19902,0.016932,-1
...,...,...,...
25532,14572,0.015237,-1
25794,1184,0.033881,-1
25974,26077,0.084729,-1
25990,22236,0.025407,-1


In [149]:
fd3 = (
            fd2.groupby(["product_type_no"])["price"]
            .mean()
            .reset_index(name="mean")
        )

In [150]:
fd3=pd.merge(fd2,fd3, on ="product_type_no",how="left")

In [152]:
fd3[fd3["product_type_no"]==262]

,article_id,price,product_type_no,mean
0,1859,0.061000,262,0.066567
184,11295,0.050831,262,0.066567
192,9070,0.027102,262,0.066567
270,3248,0.023712,262,0.066567
287,5791,0.084729,262,0.066567
...,...,...,...,...
26118,22672,0.076254,262,0.066567
26138,3622,0.040661,262,0.066567
26184,25783,0.067780,262,0.066567
26201,10317,0.067780,262,0.066567


In [154]:
fd4 = fd2.groupby(["product_type_no"])["price"].min().reset_index(name="min")
fd4=pd.merge(fd3, fd4, on=["product_type_no"], how='left')

In [155]:
fd4[fd4["product_type_no"]==262]

,article_id,price,product_type_no,mean,min
0,1859,0.061000,262,0.066567,0.006763
184,11295,0.050831,262,0.066567,0.006763
192,9070,0.027102,262,0.066567,0.006763
270,3248,0.023712,262,0.066567,0.006763
287,5791,0.084729,262,0.066567,0.006763
...,...,...,...,...,...
26118,22672,0.076254,262,0.066567,0.006763
26138,3622,0.040661,262,0.066567,0.006763
26184,25783,0.067780,262,0.066567,0.006763
26201,10317,0.067780,262,0.066567,0.006763


In [157]:
fd5 = fd2.groupby(["product_type_no"])["price"].max().reset_index(name="max")
fd5=pd.merge(fd4, fd5, on=["product_type_no"], how='left')

In [159]:
fd5[fd5["product_type_no"]==262]

,article_id,price,product_type_no,mean,min,max
0,1859,0.061000,262,0.066567,0.006763,0.337288
184,11295,0.050831,262,0.066567,0.006763,0.337288
192,9070,0.027102,262,0.066567,0.006763,0.337288
270,3248,0.023712,262,0.066567,0.006763,0.337288
287,5791,0.084729,262,0.066567,0.006763,0.337288
...,...,...,...,...,...,...
26118,22672,0.076254,262,0.066567,0.006763,0.337288
26138,3622,0.040661,262,0.066567,0.006763,0.337288
26184,25783,0.067780,262,0.066567,0.006763,0.337288
26201,10317,0.067780,262,0.066567,0.006763,0.337288


In [160]:
fd5["price_mean"]=fd5["price"]/fd5["mean"]
fd5["price_min"]=fd5["price"]/fd5["min"]
fd5["price_max"]=fd5["price"]/fd5["max"]

In [161]:
fd5

,article_id,price,product_type_no,mean,min,max,price_mean,price_min,price_max
0,1859,0.061000,262,0.066567,0.006763,0.337288,0.916370,9.020050,0.180854
1,4060,0.016932,272,0.036057,0.002695,0.422034,0.469596,6.283019,0.040120
2,457,0.011847,298,0.019276,0.001678,0.050831,0.614634,7.060606,0.233078
3,8839,0.018627,272,0.036057,0.002695,0.422034,0.516603,6.911950,0.044137
4,5072,0.015237,273,0.026186,0.003373,0.422034,0.581880,4.517588,0.036104
...,...,...,...,...,...,...,...,...,...
26247,16131,0.022017,265,0.036440,0.002627,0.506780,0.604191,8.380645,0.043445
26248,15180,0.059305,258,0.028166,0.001508,0.422034,2.105547,39.314607,0.140522
26249,3590,0.010153,306,0.022722,0.002695,0.050831,0.446809,3.767296,0.199733
26250,17935,0.025407,306,0.022722,0.002695,0.050831,1.118140,9.427673,0.499833


In [162]:
feature=fd5[[DEFAULT_ITEM_COL,"price_mean","price_min","price_max"]].drop_duplicates()

In [163]:
feature

,article_id,price_mean,price_min,price_max
0,1859,0.916370,9.020050,0.180854
1,4060,0.469596,6.283019,0.040120
2,457,0.614634,7.060606,0.233078
3,8839,0.516603,6.911950,0.044137
4,5072,0.581880,4.517588,0.036104
...,...,...,...,...
26247,16131,0.604191,8.380645,0.043445
26248,15180,2.105547,39.314607,0.140522
26249,3590,0.446809,3.767296,0.199733
26250,17935,1.118140,9.427673,0.499833


In [7]:
fd1=fd.groupby([DEFAULT_USER_COL, 't_dat']).size().reset_index(name='counts')

In [8]:
fd1

,customer_id,t_dat,counts
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,3
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,1
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,2
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,1
...,...,...,...
9080174,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-05,3
9080175,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-09,2
9080176,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-25,1
9080177,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-06-22,1


In [10]:
fd2=fd1.groupby([DEFAULT_USER_COL])["counts"].mean().reset_index(name='average')

In [11]:
fd2

,customer_id,average
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2.100000
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.739130
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.571429
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667
...,...,...
1362276,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,4.636364
1362277,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,4.421053
1362278,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,2.368421
1362279,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.750000


In [12]:
intervals = [("2020-08-01", "2020-08-31"), ("2019-08-01", "2019-09-30"), ("2018-08-01", "2018-09-30")]

In [16]:
m = np.logical_or.reduce([np.logical_and(fd['t_dat'] >= l, fd['t_dat'] <= u)
                                    for l, u in intervals])
out = fd.loc[m]

In [17]:
out

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2
...,...,...,...,...,...
30990050,2020-08-31,fff7e7674509592818bf453391af43a85eaaac9a52d858...,0658030020,0.050831,1
30990051,2020-08-31,fff8722f6fe413c2df38cb09da2f458e5efe65cbcb3d08...,0923643001,0.025407,2
30990052,2020-08-31,fff8722f6fe413c2df38cb09da2f458e5efe65cbcb3d08...,0894320001,0.033881,2
30990053,2020-08-31,fffd176916f8c790ad54335c0f8478e613b5cb9b09ce2c...,0873678002,0.025407,1


In [79]:
df=fd.drop_duplicates(["t_dat",DEFAULT_USER_COL])

In [80]:
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
6,2018-09-20,4,4,0.032186,2
...,...,...,...,...,...
14361074,2020-09-22,59779,20150,0.050831,2
14361083,2020-09-22,198005,20308,0.059305,2
14361085,2020-09-22,798833,21707,0.043203,1
14361086,2020-09-22,395317,4994,0.006763,1


In [81]:
df['date_diff'] = (df.pop('t_dat').groupby(df['customer_id']).diff().dt.days.fillna(0, downcast='infer'))

/tmp/ipykernel_23885/1392639203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_diff'] = (df.pop('t_dat').groupby(df['customer_id']).diff().dt.days.fillna(0, downcast='infer'))


In [82]:
df

,customer_id,article_id,price,sales_channel_id,date_diff
0,0,0,0.008458,2,0
2,1,1,0.033881,1,0
3,2,2,0.021593,1,0
4,3,3,0.022017,2,0
6,4,4,0.032186,2,0
...,...,...,...,...,...
14361074,59779,20150,0.050831,2,17
14361083,198005,20308,0.059305,2,2
14361085,798833,21707,0.043203,1,21
14361086,395317,4994,0.006763,1,11


In [85]:
df[df[DEFAULT_USER_COL]==0]

,customer_id,article_id,price,sales_channel_id,date_diff
0,0,0,0.008458,2,0
3102169,0,0,0.007610,2,319
3265030,0,644,0.039576,2,11
4089565,0,1952,0.030492,2,51
8123440,0,1482,0.033881,2,192


In [83]:
user_average=df.groupby(DEFAULT_USER_COL)["date_diff"].mean().reset_index()

In [84]:
user_average

,customer_id,date_diff
0,0,114.600000
1,1,53.230769
2,2,67.125000
3,3,124.333333
4,4,0.000000
...,...,...
1136201,1136201,0.000000
1136202,1136202,0.000000
1136203,1136203,0.000000
1136204,1136204,0.000000


In [63]:
count_mb=fd.groupby(DEFAULT_USER_COL)["sales_channel_id"].size().reset_index()

In [64]:
count_mb

,customer_id,sales_channel_id
0,0,14
1,1,24
2,2,26
3,3,7
4,4,1
...,...,...
1136201,1136201,2
1136202,1136202,1
1136203,1136203,3
1136204,1136204,2


In [65]:
df=fd[fd["sales_channel_id"]==1]

In [66]:
count_mb1=df.groupby(DEFAULT_USER_COL)["sales_channel_id"].size().reset_index()

In [70]:
df.groupby(DEFAULT_USER_COL)["sales_channel_id"].size()

customer_id
1           7
2          26
5           2
6          18
7           9
           ..
1136190     6
1136194     7
1136196     1
1136197     2
1136202     1
Name: sales_channel_id, Length: 610744, dtype: int64

In [67]:
count=pd.merge(count_mb,count_mb1, on =DEFAULT_USER_COL, how='left')

In [68]:
count

,customer_id,sales_channel_id_x,sales_channel_id_y
0,0,14,NaN
1,1,24,7.0
2,2,26,26.0
3,3,7,NaN
4,4,1,NaN
...,...,...,...
1136201,1136201,2,NaN
1136202,1136202,1,1.0
1136203,1136203,3,NaN
1136204,1136204,2,NaN


In [74]:
count=count.fillna(0)

,customer_id,sales_channel_id_x,sales_channel_id_y,sales_channel_score
0,0,14,0.0,0.000000
1,1,24,7.0,0.291667
2,2,26,26.0,1.000000
3,3,7,0.0,0.000000
4,4,1,0.0,0.000000
...,...,...,...,...
1136201,1136201,2,0.0,0.000000
1136202,1136202,1,1.0,1.000000
1136203,1136203,3,0.0,0.000000
1136204,1136204,2,0.0,0.000000


In [72]:
count["sales_channel_score"]=count["sales_channel_id_y"]/count["sales_channel_id_x"]

In [73]:
count

,customer_id,sales_channel_id_x,sales_channel_id_y,sales_channel_score
0,0,14,NaN,NaN
1,1,24,7.0,0.291667
2,2,26,26.0,1.000000
3,3,7,NaN,NaN
4,4,1,NaN,NaN
...,...,...,...,...
1136201,1136201,2,NaN,NaN
1136202,1136202,1,1.0,1.000000
1136203,1136203,3,NaN,NaN
1136204,1136204,2,NaN,NaN


In [40]:
count_mb1

customer_id
1           7
2          26
5           2
6          18
7           9
           ..
1136190     6
1136194     7
1136196     1
1136197     2
1136202     1
Name: sales_channel_id, Length: 610744, dtype: int64

In [ ]:
fd[DEFAULT_USER_COL].nunique()

In [ ]:
fd = fd.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
fd

In [ ]:
count_mb = fd.groupby(DEFAULT_ITEM_COL).count()
feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": "popularity"})

In [ ]:
feature

In [ ]:
feature["popularity_score"]=(feature["popularity"]-feature["popularity"].min())/(feature["popularity"].max()-feature["popularity"].min())

In [ ]:
feature

In [ ]:
feature['rank'] = feature['popularity_score'].rank(ascending=False,method='min').astype(int)

In [ ]:
feature

In [ ]:
df=dr.get_filtered_full_data()

In [ ]:
df= df.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
df

In [ ]:
df["last_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
df["first_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
df["time_score"] = (df["t_dat"] - df["first_buy"]) / (
            df["last_buy"] - df["first_buy"]
        )

In [ ]:
df=df.fillna(1)#df["tdiff"] =1/ ((df["last_buy"]-df["t_dat"]).dt.days) 

In [ ]:
df

In [ ]:
df["rank_time"] = df.groupby(DEFAULT_USER_COL)["time_score"].rank(ascending=False,method='min').astype(int)

In [ ]:
df

In [ ]:
df.loc[df[DEFAULT_USER_COL]==1]

In [ ]:
feature

In [ ]:
final=pd.merge(df, feature, on =DEFAULT_ITEM_COL, how='left')

In [ ]:
final

In [ ]:
final.loc[final[DEFAULT_USER_COL]==1]

In [ ]:
user=dr.get_filtered_full_data()

In [ ]:
user = user.drop_duplicates([DEFAULT_USER_COL])

In [ ]:
user['temp']=1

In [ ]:
user=user[[DEFAULT_USER_COL,'temp']]

In [ ]:
user

In [ ]:
feature_100=feature[feature['rank']<=100]

In [ ]:
feature_100['temp']=1

In [ ]:
final1=pd.merge(user, feature_100, on='temp')

In [ ]:
final1

In [ ]:
final1=final1.drop('temp', axis=1)

In [ ]:
final1

In [ ]:
final1[final1['customer_id']==1]

In [ ]:
final

In [ ]:
final=final.drop(['popularity_score','rank','popularity','t_dat','price','sales_channel_id','last_buy','first_buy'], axis=1)

In [ ]:
final2=pd.merge(final,final1,on=[DEFAULT_USER_COL,DEFAULT_ITEM_COL],how='outer')

In [ ]:
final[final[DEFAULT_USER_COL]==1]

In [ ]:
final2[final2[DEFAULT_USER_COL]==1]

In [ ]:
mask = final2.isnull().any(axis=1)

In [ ]:
final3=final2[~mask]

In [ ]:
final3[final3[DEFAULT_USER_COL]==1]

In [ ]:
final2=final2.fillna(0)

In [ ]:
final2

In [ ]:
final2=final2.drop(['rank_time','rank'],axis=1)

In [ ]:
a=0.95
eps=1e-6
final2['score']=a*final2['time_score']+(1-a+eps)*final2['popularity_score']

In [ ]:
final2

In [ ]:
final2["rank"] = final2.groupby(DEFAULT_USER_COL)["score"].rank(ascending=False,method='min').astype(int)

In [ ]:
final2

In [ ]:
final2=final2.drop(["time_score","popularity","popularity_score"],axis=1)

In [ ]:
final2[DEFAULT_USER_COL].nunique()

In [ ]:
final2

In [ ]:
cutoff = final2.groupby(DEFAULT_USER_COL).size().values

In [ ]:
cutoff

In [ ]:
i=0
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=i+cut

In [ ]:
filter_indices

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values(DEFAULT_USER_COL, ascending=True)

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values([DEFAULT_USER_COL, "rank"], ascending=True)

In [ ]:
final2

In [ ]:
final2 = final2.reset_index(drop=True)

In [ ]:
final_df = final2.loc[filter_indices]

In [ ]:
final_df[DEFAULT_USER_COL].nunique()

In [ ]:
final_final_df = final_df.drop(["score","rank"] ,axis=1)

In [ ]:
final_final_df[DEFAULT_USER_COL].nunique()

In [ ]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()     

In [ ]:
sh.create_submission_filtered_data([final_final_df], sub_name="Time_Pop")